In [4]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import json
import math
import pandas as pd

from data_utils import MLDataset, collate_fn
from new_modeling import * # You can import your custom model classes from modeling.py


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\CafeAlle\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\CafeAlle\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\CafeAlle\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    s

In [5]:
assert torch.cuda.is_available()

# Use 0th GPU for training
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# You can add or modify your Seq2SeqModel's hyperparameter (keys and values)
kwargs = {
    'hidden_dim': 256,
    'n_rnn_layers': 1,
    'rnn_dropout': 0.5,
    'embedding_dim': 90

    #teacherforcing ratio
}
kwargs_generate = {
    # you can add arguments for your model's generate function
    'max_length': 12
}
BATCH_SIZE = 128
NUM_CLASSES = 28

In [7]:
# You can use your own model class and model path

model = Seq2SeqModelChallenge(num_classes=NUM_CLASSES, **kwargs).to(device)
print(model)

model_path = './challenge_model.pt'

c:\Users\CafeAlle\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2SeqModelChallenge(
  (encoder): Encoder(
    (cnn): CustomCNN(
      (conv1): Sequential(
        (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (conv2): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (conv3): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=Fa

In [8]:
# Do not modify this cell!

test_ds = MLDataset('data_final/imgs/test', 'data_final/labels/test_dummy.json')
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

state = torch.load(model_path)
model.load_state_dict(state["model"])
model.eval()

ids = []
predictions = []
for batch_idx, (data, _, lengths) in enumerate(tqdm(test_dl)):       
    data = data.to(device) # (B, T, H, W, C)
    
    # start tokens should be located at the first position of the decoder input
    start_tokens = (torch.ones([data.size(0), 1]) * 27).to(torch.long).to(device)
    with torch.no_grad():
        generated_tok = model.generate(data, lengths, start_tokens, **kwargs_generate) # (B, T)

    for i in range(generated_tok.size(0)):
        sample_idx = batch_idx * BATCH_SIZE + i + 1
        ids.append(sample_idx)
        pred = 0
        for j, tok in enumerate(generated_tok[i][:lengths[i].int()].tolist()):
            pred += tok * math.pow(28, j)
        predictions.append(int(pred))

sub_df = pd.DataFrame({
    'id': ids,
    'prediction': predictions
})

sub_df.to_csv('submission.csv', index=False)
print("Created submission file successfully!")

100%|██████████| 69/69 [04:14<00:00,  3.69s/it]

Created submission file successfully!
